### Imports

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils
from tqdm import tqdm
from scipy.special import erf
import scipy.stats

from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from dmipy.core.modeling_framework import MultiCompartmentSphericalMeanModel
from dmipy.signal_models import sphere_models, cylinder_models, gaussian_models

from scipy.io import savemat

### Calculate ADC' (Prime)

In [31]:
def calc_adc_prime(adc, sigma, axr, tm):
   adc_prime = adc * (1 - sigma* np.exp(-tm*axr))
   return adc_prime


### Simulate Signal

In [32]:
def simulate_sig(adc, sigma, axr, bf, be, tm, acq):
    """Generate an output signal s based on known inputs for a given voxel
Inputs  - adc:      apparent diffusion coefficient [m2/s]
        - sigma:    filter efficiency
        - axr:      exchange rate [1/s]
        - bf:       filter block b-value [m2/s]
        - be:       encoding block b-value [m2/s]
        - tm:       mixing time [s]

Output: - s:        signal (sum of the magnetisations) single value
   Based off Elizabeth's code
    """

    if bf == 0 and tm[acq] == min(tm):
        tm[acq] = np.inf

    #calculate ADC as fnc of mixing time

    adc_prime = calc_adc_prime(adc,sigma,axr,tm[acq])

    #compute signal
    signal = np.exp(-adc_prime*be)

    return signal, adc_prime
    

### Running functions

In [33]:
# example code for Gabe
# E Powell, 24/11/2023

nvox = 100 # number of voxels to simulate

bf = np.array([0, 0, 250, 250, 250, 250, 250, 250]) * 1e6   # filter b-values [s/m2]
be = np.array([0, 250, 0, 250, 0, 250, 0, 250]) * 1e6       # encoding b-values [s/m2]
tm = np.array([20, 20, 20, 20, 200, 200, 400, 400]) * 1e-3  # mixing time [s]

sim_adc = np.random.uniform(1e-9,1e-10,nvox)                # ADC, simulated [m2/s]
sim_sigma = np.random.uniform(0.1,0.9,nvox)                 # sigma, simulated [a.u.]
sim_axr = np.random.uniform(1,5,nvox)                       # AXR, simulated [s-1]

# simulate signals    
sigs_and_axr_prime = np.array([[simulate_sig(sim_adc[voxel], sim_sigma[voxel], sim_axr[voxel], bf[acq], be[acq], tm, acq) 
                    for acq in range(np.size(tm))] 
                    for voxel in range(nvox)])

signals = sigs_and_axr_prime[:, :, 0]
axr_prime = sigs_and_axr_prime[:, :, 1]





0.0
